In [1]:
import Statistics

In [2]:
# Generate data
features = rand(100,4)
target = randn(100)
target .= target .>= 0 
data = [features  target]

100×5 Matrix{Float64}:
 0.435855   0.552138   0.35017     0.625914     1.0
 0.418316   0.55801    0.970549    0.028202     0.0
 0.0425511  0.106551   0.80239     0.0575727    0.0
 0.96017    0.259848   0.130008    0.976833     1.0
 0.866035   0.0716926  0.426494    0.42161      0.0
 0.507702   0.698847   0.831157    0.144794     1.0
 0.235633   0.279742   0.0253972   0.296214     0.0
 0.14155    0.450073   0.861429    0.790462     0.0
 0.0989363  0.259983   0.782257    0.594284     0.0
 0.749966   0.600157   0.388327    0.143786     0.0
 ⋮                                              
 0.122784   0.640284   0.00588085  0.502691     1.0
 0.0989226  0.981694   0.0239282   0.000287068  0.0
 0.938865   0.404156   0.990087    0.600349     0.0
 0.857281   0.166109   0.935074    0.344205     1.0
 0.210239   0.40311    0.665651    0.881559     1.0
 0.942845   0.267881   0.957587    0.581308     1.0
 0.462905   0.37775    0.826841    0.147573     1.0
 0.332526   0.834125   0.193823    0.402316 

In [3]:
size(features)

(100, 4)

zip([1 2 … 5 6], [2 3 … 6 7])

In [158]:
function seprate_classes(x,y) # Separate the dataset into a subset of data for each class
    seprated_classes = Dict()
    for i in 1:size(x,1)
        feature = x[i,:]
        class_name = y[i]
        if class_name ∉ keys(seprated_classes) 
            seprated_classes[class_name] = []
        end
        append!(seprated_classes[class_name],feature)
    end
    
    return seprated_classes
end

seprate_classes (generic function with 1 method)

In [159]:
classes = seprate_classes(features,target)
classes[1]

212-element Vector{Any}:
 0.4358546386007687
 0.5521383077421034
 0.3501700528566135
 0.6259140007432856
 0.9601699596240566
 0.25984811643200556
 0.1300077052383739
 0.97683347823513
 0.5077020669283285
 0.698846867956817
 ⋮
 0.5813084803012523
 0.4629052566326427
 0.37775015371816434
 0.8268411893818528
 0.14757341382435352
 0.09047670002268204
 0.2627718982402032
 0.9833234906810514
 0.594998619093331

In [101]:
function std_mean(x) # Calculates standard deviation and mean of features.
    std = []
    mean = []
    for feature in eachcol(x)
        append!(std,Statistics.std(feature))
        append!(mean,Statistics.mean(feature))
    end
    return std , mean
end

std_mean (generic function with 1 method)

In [102]:
std,mean = std_mean(features)

(Any[0.30924455970810494, 0.2794567534420882, 0.30675808785680186, 0.2886211045461954], Any[0.47459656614455403, 0.44330638272600636, 0.4616431719680532, 0.5104882696072105])

In [87]:
features |> size

(100, 4)

In [60]:
[println(i) for i in std_mean(features)]

Any[0.30924455970810494, 0.2794567534420882, 0.30675808785680186, 0.2886211045461954]
Any[0.47459656614455403, 0.44330638272600636, 0.4616431719680532, 0.5104882696072105]


2-element Vector{Nothing}:
 nothing
 nothing

In [9]:
length(stdmean)

16

In [10]:
function distribution(x,std,mean) # Gaussian Distribution Function
    exponent = exp.(-((x .- mean).^2 ./ (2 .* std .^ 2)))
    return exponent ./ (sqrt(2*π)*std)
end

distribution (generic function with 1 method)

In [103]:
function fit(x,y)
    classes = seprate_classes(x,y)
    class_summary = Dict()
    for item in classes
        @show class_name , feature_val = item.first , item.second
        std,mean = std_mean(feature_val);
        summary = Dict("mean"=>mean,"std"=>std);
        class_summary[class_name] = Dict("prior_proba"=>length(feature_val)/size(x,1),
                                        "summary"=>summary)
    end
    return class_summary        
end

fit (generic function with 1 method)

In [104]:
class_summary = fit(features,target)

(class_name, feature_val) = (item.first, item.second) = 

(0.0, Any[0.41831553353563133, 0.5580097994921733, 0.9705492853043308, 0.02820201166354941, 0.04255110047912847, 0.10655093693384088, 0.8023895886201174, 0.057572728953397934, 0.8660354743238119, 0.07169257638693138, 0.42649412745923754, 0.4216104852173289, 0.23563277620033585, 0.279741873140659, 0.025397218687447154, 0.29621401171408057, 0.1415500058214001, 0.45007315807978676, 0.8614287093933103, 0.7904619800124605, 0.09893626109597176, 0.2599833360838233, 0.7822573636344272, 0.5942836940285133, 0.7499659296433462, 0.6001573888692446, 0.3883270734885411, 0.14378550677612156, 0.021980436917751978, 0.10341541127791765, 0.0009298503195360741, 0.942380898415229, 0.33033675630451775, 0.5451060487008462, 0.4930075473648774, 0.6403835631259123, 0.44496272759376776, 0.2558767301152526, 0.10517437924645046, 0.8754516854732564, 0.7821254377238146, 0.9426253451910823, 0.15580661971352106, 0.24607360756442975, 0.31645580760175795, 0.34103725424358167, 0.32914566463033135, 0.924589092294555, 0.11

Dict{Any, Any} with 2 entries:
  0.0 => Dict{String, Any}("summary"=>Dict{String, Vector{Any}}("mean"=>[0.4680…
  1.0 => Dict{String, Any}("summary"=>Dict{String, Vector{Any}}("mean"=>[0.4764…

In [105]:
class_summary[1]["summary"]

Dict{String, Vector{Any}} with 2 entries:
  "mean" => [0.476484]
  "std"  => [0.295477]

In [71]:
classes = seprate_classes(features,target)
std_mean(classes[0])
# [i for i in std_mean(class)]

(Any[0.29771757506982854], Any[0.46802516251492404])

In [13]:
function predict(x,class_summary)
    MAPs = []
    for row in eachrow(x)
        joint_proba = Dict()
        for item in class_summary
            class_name , features = item.first,item.second
            total_features = length(features["summary"])
            likelihood =1

            for idx in 1:total_features
                feature = row[idx]
                mean_ = features["summary"][idx]["mean"]
                std_ = features["summary"][idx]["std"]
                normal_proba = distribution(feature,std_,mean_)
                likelihood *= normal_proba
            end
            prior_proba = features["prior_proba"]
            joint_proba[class_name] = prior_proba * likelihood
        end
        MAP = max((x,y)->joint_proba[x]>joint_proba[y] ? x : y,keys(joint_proba)) # ?
        append!(MAPs,MAP)
    end
    return MAPs
end

predict (generic function with 1 method)

In [14]:
maps = predict(features,class_summary)

MethodError: MethodError: no method matching getindex(::String, ::String)
Closest candidates are:
  getindex(::AbstractString, !Matched::Colon) at strings/basic.jl:189
  getindex(::String, !Matched::Int64) at strings/string.jl:226
  getindex(::AbstractString, !Matched::Integer) at strings/basic.jl:184
  ...

In [73]:
maps = []
for row in eachrow(features)
    for item in class_summary
        class_name , features_val = item.first,item.second
        # println(class_name)
        println(features_val)
        total_features = length(features_val["summary"])
        likelihood =1
        
        println("doroste")
        for idx in 1:total_features
            @show feature = row[idx]
            mean_ = features_val["summary"][idx]["mean"]
            std_ = features_val["summary"][idx]["std"]
            normal_proba = distribution(feature,std_,mean_)
            likelihood *= normal_proba
        end
        prior_proba = features_val["prior_proba"]
        joint_proba[class_name] = prior_proba * likelihood

        println("==============")
    end
end
    

Dict{String, Any}("summary" => Dict{String, Vector{Any}}("mean" => [0.46802516251492404, 0.47648447401781435], "std" => [0.29771757506982854, 0.29547705058560075]), "prior_proba" => 1.88)
doroste
feature = row[idx] = 0.4358546386007687


KeyError: KeyError: key 1 not found

In [93]:
class_summary[1]["summary"]["mean"]

212-element Vector{Any}:
 0.4358546386007687
 0.5521383077421034
 0.3501700528566135
 0.6259140007432856
 0.9601699596240566
 0.25984811643200556
 0.1300077052383739
 0.97683347823513
 0.5077020669283285
 0.698846867956817
 ⋮
 0.5813084803012523
 0.4629052566326427
 0.37775015371816434
 0.8268411893818528
 0.14757341382435352
 0.09047670002268204
 0.2627718982402032
 0.9833234906810514
 0.594998619093331

In [17]:
[item.first=>item.second[i] for (i,item) in enumerate(a)]

2-element Vector{Pair{String, Int64}}:
 "wwww" => 6
 "qqqq" => 2

In [18]:
for key in keys(a)
    for (i,item) in enumerate(a[key])
        println(key=>item)
    end
end

"wwww" => 6
"wwww" => 7
"wwww" => 8
"wwww" => 9
"qqqq" => 1
"qqqq" => 2
"qqqq" => 3
"qqqq" => 4


In [19]:
[i=>j for i in 1:5,j in 1:5]

5×5 Matrix{Pair{Int64, Int64}}:
 1=>1  1=>2  1=>3  1=>4  1=>5
 2=>1  2=>2  2=>3  2=>4  2=>5
 3=>1  3=>2  3=>3  3=>4  3=>5
 4=>1  4=>2  4=>3  4=>4  4=>5
 5=>1  5=>2  5=>3  5=>4  5=>5